In [37]:
import numpy as np
from numpy.linalg import eig
import pandas as pd
import py_lets_be_quickly_rational
import time
from scipy import interpolate

0) intro
1) review > only optimal stopping
2) model > stoch correl; no resim because chance player always the same; but exploration question remains; check with deepmind paper
2) geodesic+grad descent
3) dqn
4) mcts

In [2]:
# s0, v0, kappa, theta, alpha=vov, rho
modelParam = np.array([
    [1.0, 0.2**2, 4.0, 0.25**2, 0.5, -0.5],
    [1.0, 0.25**2, 3.0, 0.35**2, 0.4, -0.6],
    [1.0, 0.3**2, 2.0, 0.3**2, 0.3, -0.7],
])

n = len(modelParam)

# constraint 2 theta vbar > alpha**2
for i in range(n):
    _, _, theta, vbar, alpha, _ = modelParam[i]
    print(2*theta*vbar-alpha**2)

0.25
0.5749999999999998
0.27


In [3]:
C = np.array([
    [1.0, 0.7, 0.3],
    [0.7, 1.0, 0.5],
    [0.3, 0.5, 1.0]
])

L = np.linalg.cholesky(C)
print(C-np.matmul(L, L.T))
    
for i in range(3):
    b=np.random.multivariate_normal(np.zeros(3),0.02**2*np.eye(3))
    a = L[i,:] + b
    L[i,:] = a / np.linalg.norm(a)
print(np.matmul(L, L.T))

[[ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  1.11022302e-16  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00 -2.22044605e-16]]
[[1.         0.69051445 0.29834039]
 [0.69051445 1.         0.49906237]
 [0.29834039 0.49906237 1.        ]]


In [4]:
def xi(a,b,c,d,k):
    r1 = np.cos(k*np.pi*(d-a)/(b-a))*np.exp(d) - np.cos(k*np.pi*(c-a)/(b-a))*np.exp(c)
    r2 = k*np.pi/(b-a)*np.sin(k*np.pi*(d-a)/(b-a))*np.exp(d) - k*np.pi/(b-a)*np.sin(k*np.pi*(c-a)/(b-a))*np.exp(c)
    return (r1+r2) / (1+(k*np.pi/(b-a))**2)

def vpsi(a,b,c,d,ks):
    ks = np.maximum(ks,1)
    r = (np.sin(ks*np.pi*(d-a)/(b-a)) - np.sin(ks*np.pi*(c-a)/(b-a)))*(b-a)/(ks*np.pi)
    r[0] = d-c
    return r

def vv_put_orig(a, b, ks):
    r = 2 / (b-a) * (-xi(a,b,a,0,ks)+vpsi(a,b,a,0,ks))
    r[0] = r[0] / 2.0
    return r

def phi_heston2(omega, kappa,theta, alpha, rho,V0,T):
    z=omega
    tau=T
    v=V0
    vbar=theta
    lamb=kappa
    eta=alpha
    a0=lamb-rho*eta*z*1j
    gamma=np.power(eta**2*(z**2+z*1j)+a0*a0, 0.5)
    G=(a0-gamma)/(a0+gamma)
    a1=v/eta/eta*((1-np.exp(-gamma*tau))/(1-G*np.exp(-gamma*tau)))*(a0-gamma)
    a2=lamb*vbar/eta/eta*(tau*(a0-gamma)-2*np.log((1-G*np.exp(-gamma*tau))/(1-G))) #c*vbar
    return np.exp(a1+a2)

def cumulants(kappa, theta, V0, t):
    c1 = (1-np.exp(-kappa*t))*(theta-V0)/(2*kappa)-theta*t/2
    c21 = V0 / (4*kappa**3) * (4*kappa**2*(1+(rho*alpha*t-1)*np.exp(-kappa*t))
                             +kappa*(4*rho*alpha*(np.exp(-kappa*t)-1)-2*alpha**2*t*np.exp(-kappa*t))
                             +alpha**2*(1-np.exp(-2*kappa*t)))
    c22 = theta / (8*kappa**3) * (8*kappa**3*t - 8*kappa**2*(1+rho*alpha*t+(rho*alpha*t-1)*np.exp(-kappa*t))+\
                                  2*kappa*((1+2*np.exp(-kappa*t))*alpha**2*t+ 8*(1-np.exp(-kappa*t))*rho*alpha)+\
                                  alpha**2*(np.exp(-2*kappa*t)+4*np.exp(-kappa*t)-5))
    return c1, c21+c22

In [5]:
T=1
S0, V0, kappa, theta, alpha, rho = modelParam[1,:]
c1, c2 = cumulants(kappa, theta, V0, T)
L=12
a=c1 - L*c2**0.5
b=c1 + L*c2**0.5
F=1
N=50

In [12]:
Ks=np.linspace(0.95,1.05,11)
t1=time.perf_counter()
xs=np.log(F/Ks)
ks = np.array(range(N))
xxs = ks*np.pi/(b-a)

h = phi_heston2(xxs, kappa,theta,alpha,rho,V0,T) * vv_put_orig(a,b,ks)
res = np.zeros(len(Ks))
iv = np.zeros(len(Ks))
q=-1 #put
numIt=1
for i in range(len(xs)):
    res[i] = Ks[i] * np.sum(h * np.exp(1j * (xs[i] - a) * xxs)).real
    iv[i] = py_lets_be_quickly_rational.implied_volatility_from_a_transformed_rational_guess_with_limited_iterations(res[i], S0, Ks[i], T, q, numIt)
t2=time.perf_counter()
print("time:", t2-t1)

for i in range(len(Ks)):
    print(Ks[i], res[i])
print("calls")
for i in range(len(Ks)):
    print(Ks[i], res[i] + 1 - Ks[i])

time: 0.0009815669982344843
0.95 0.09968511605031086
0.96 0.1044186612554525
0.97 0.10928112956123624
0.98 0.11427204852082093
0.99 0.11939082087068936
1.0 0.12463672787430953
1.01 0.13000893294040944
1.02 0.13550648549460667
1.03 0.14112832508270606
1.04 0.14687328568367972
1.05 0.15274010021014287
calls
0.95 0.14968511605031098
0.96 0.14441866125545255
0.97 0.13928112956123617
0.98 0.13427204852082086
0.99 0.12939082087068932
1.0 0.12463672787430946
1.01 0.12000893294040949
1.02 0.11550648549460663
1.03 0.11112832508270598
1.04 0.1068732856836796
1.05 0.1027401002101429


In [20]:
T=2
n_reset=8
n_resetPerYear=n_reset//T
n_simStepsPerReset=365//(n_resetPerYear)
n_simStepsPerYear=n_simStepsPerReset*n_resetPerYear
nt=n_simStepsPerReset*n_reset
T_reset=[n_simStepsPerReset*q for q in range(1,n_reset)]

dt=1.0/n_simStepsPerYear
sqrtDt=dt**0.5
na=C.shape[0]
ns=10000
volCorrel=0.02

In [21]:
#simulate correl
U0 = np.linalg.cholesky(C).T
U0 = np.repeat(U0[np.newaxis, :, :], ns, axis=0)
U = np.repeat(U0[np.newaxis, :, :, :], n_reset, axis=0)
for i in range(n_reset):
    A=U0 if i==0 else U[i-1,:,:,:]
    A=A + np.random.multivariate_normal(np.zeros(na),volCorrel**2*np.eye(na),size=(ns,na))
    U[i,:,:,:]=A/np.linalg.norm(A,axis=2)[:,:,np.newaxis]
#np.matmul(U[-1,0,:,:], U[-1,0,:,:].T)

In [23]:
W=np.random.normal(size=(n_reset,ns,na,n_simStepsPerReset))
print(U.shape, W.shape)
W1=np.matmul(U,W)
print(W1.shape)

(8, 10000, 3, 3) (8, 10000, 3, 91)
(8, 10000, 3, 91)


In [25]:
rho=np.ones_like(W1)
for i in range(na):
    rho[:,:,i,:]=modelParam[i,-1]
W2 = rho*W1 + np.sqrt(1-np.square(rho)) * np.random.normal(size=(n_reset,ns,na,n_simStepsPerReset))

In [26]:
X=np.zeros((n_reset,ns,na))
V=np.zeros((n_reset,ns,na))

Xr=np.tile(np.log(modelParam[:,0]),(ns,1))
Vr=np.tile(modelParam[:,1],(ns,1))

for i in range(n_reset):
    for j in range(n_simStepsPerReset):
        sqrtV = np.sqrt(Vr)
        Xr = Xr + sqrtV * sqrtDt * W1[i,:,:,j] - 0.5 * Vr * dt
        Vr = np.abs(modelParam[:,2]*(modelParam[:,3]-Vr)*dt + modelParam[:,4]*sqrtV*sqrtDt*W2[i,:,:,j])         
    X[i,:,:] = Xr
    V[i,:,:] = Vr
S=np.exp(X)

In [27]:
np.mean(S[-1,:,:],axis=0)

array([0.99952614, 0.99994377, 0.99987206])

In [35]:
nk=9
vol_ref = np.sqrt(modelParam[:,3])
ms = np.array([-5,-3, -2.5,-2, -1.5,-1, -0.75,-0.5, -0.25, 0, 0.25, 0.5, 0.75, 1, 1.5, 2, 2.5, 3, 5])
periodDt = T/n_reset
sqrtPeriodDt=periodDt**0.5

Ks = np.zeros((n_reset-1,ns,na,nk))
prices = np.zeros((n_reset-1,ns,na,nk))
vols = np.zeros((n_reset-1,ns,na,nk))
for i in range(n_reset-1):
    for j in range(2): #ns
        Ks[i,j,:,:]=S[i,j,:].reshape((na,1))*np.exp(np.matmul(vol_ref.reshape((na,1)),ms.reshape((1,-1)))*sqrtPeriodDt)
        for k in range(na):
            Sk = S[i,j,k]
            Vk = V[i,j,k]
            kappa, theta, alpha, rho = modelParam[k,2:]
            c1, c2 = cumulants(kappa, theta, Vk, periodDt)
            L=12
            a=c1 - L*c2**0.5
            b=c1 + L*c2**0.5
            F=Sk
            
            xs=np.log(F/Ks[i,j,k,:])
            ks = np.array(range(N))
            xxs = ks*np.pi/(b-a)

            h = phi_heston2(xxs, kappa,theta,alpha,rho,Vk, periodDt) * vv_put_orig(a,b,ks)
            res = np.zeros(len(Ks))
            iv = np.zeros(len(Ks))
            q=-1 #put
            numIt=1
            for l in range(len(xs)):
                prices[i,j,k,l] = Ks[l] * np.sum(h * np.exp(1j * (xs[l] - a) * xxs)).real
                vols[i,j,k,l] = py_lets_be_quickly_rational.implied_volatility_from_a_transformed_rational_guess_with_limited_iterations(
                    prices[i,j,k,l], Sk, Ks[l], periodDt, q, numIt)

In [ ]:
parametrisation policy
w, s0, v0, vols, correls


In [ ]:
actions:
    cts too big a step
    increase one, decrease one
    5 linear function

In [ ]:
sigma_atm
A=U[n_rest_i, path_i,:,:] >  corr
A = A * eye(sigma_atm t**0.5)
Si(0)
Sbar=sum wi Si
kappa=ln(K/Sbar)
omegai=wi Si / Sbar
Omega = eye(omegai)
zstar=
Kstar=exp(A*zstar)
sigmai
levy formula

In [ ]:
# observations = 
# w1,..,wn,
# S1,..,Sn,
# V1,..Vn,
# sigma1(ms[0]),..sigma1(ms[-1]),...sigman(ms[-1]),
# rho12,..rho1n,rho23,..rho(n-1)n
# mit n = na

K=1
nsi = 0
nri = 0

sigma_atm = np.zeros(na)
m_mid=ms//2
for i in range(na):
    sigma_atm[i] = obs[3*na+m_mid+i*len(ms)]
# nein, nicht A berechnen von correlations. 
AVar = np.matmul(A, sqrtPeriodDt*np.diag(sigma_atm))    
w = obs[:na]
S = obs[na:2*na]
Sbar = np.dot(w, S)
kappa = math.log(K / Sbar)
omega = w*S/Sbar
Omega=np.diag(omega)
C=np.ones((na, na))
c=0
for i in range(1, na):
    C[i-1,i:]=obs(na*(3+len(ms))+c:na*(3+len(ms))+c+na-i)
    C[i:,i-1]=C[i-1,i:].T
    c+=na-i
omegaComega = np.matmul(omega.T, np.matmul(C, omega))
zstar=kappa*np.matmul(A.T, omega) / omegaComega * ( 1 + kappa/2 *
    (np.eye(na) + 2 * np.matmul(A.T, np.matmul(Omega, A)) / omegaComega -
    3*np.matmul(omega.T,npmatmul(C, np.matmul(Omega, np.matmul(C, omega)))) / omegaComega**2 * np.eye(na) ))
Kstar=S*np.exp(np.matmul(A, zstar))
sigma=np.zeros(na)
for i in range(na):
    sigma[i] = interpolate.interp1d(Ks[nri,nsi,i,:], obs[3*na+i*len(ms):3*na+(i+1)*len(ms)])(Kstar[i])


In [ ]:
https://silo.tips/download/an-analysis-of-pricing-methods-for-baskets-options